# SCENIC analysis - Template script

*Last updated*: 22 jan 2019

(For a detailed tutorial with explanations of each step see: https://pyscenic.readthedocs.io )


## Before starting: Data loading/formatting and gene filtering

Set dataset location & organism. In this example we are using:

*Dataset*: __Mouse brain__ (somatosensory cortex and hippocampus); 3005 cells.

```
wget ftp://ftp.ncbi.nlm.nih.gov/geo/series/GSE60nnn/GSE60361/suppl/GSE60361%5FC1%2D3005%2DExpression%2Etxt%2Egz ; gzip -d GSE60361_C1-3005-Expression.txt.gz
```

> A. Zeisel, A. B. Moz-Manchado, S. Codeluppi, P. Lönnerberg, G. L. Manno, A. Juréus, S. Marques, H. Munguba, L. He, C. Betsholtz, C. Rolny, G. Castelo-Branco, J. Hjerling-Leffler, and S. Linnarsson, “Cell types in the mouse cortex and hippocampus revealed by single-cell RNA-seq,” Science, vol. 347, no. 6226, pp. 1138–1142, Mar. 2015.


In [7]:
exprMat_path = 'GSE60361_C1-3005-Expression.txt'
exprMat_filtered_path = 'filteredExpressionMatrix.loom' # output file, choose name...

ORGANISM = 'mouse' # Select from: 'human' / 'mouse'/ 'drosophila'
DATABASE_FOLDER = "databases/"

A motif database for the relevant organism will be used to check for existing genes.

Edit for using a different version (all databases of same version/organism contain the same genes)

In [8]:
import os

if(ORGANISM == 'human'):
    DATABASES_GLOB = os.path.join(DATABASE_FOLDER, "hg19-500bp-upstream-7species.mc9nr.feather")
if(ORGANISM == 'mouse'):
    DATABASES_GLOB = os.path.join(DATABASE_FOLDER, "mm9-500bp-upstream-7species.mc9nr.feather")
if(ORGANISM == 'drosophila'):
    DATABASES_GLOB = os.path.join(DATABASE_FOLDER, "dm6-5kb-upstream-full-tx-11species.mc8nr.feather")
    
print(DATABASES_GLOB)

databases/mm9-500bp-upstream-7species.mc9nr.feather


### Load matrix

The expression matrix of this example is stored in a text file (as tab separated matrix).

To load 10X expression files (e.g. CellRanger output in `.h5`), scripts are available at their website (https://support.10xgenomics.com/single-cell-gene-expression/software/pipelines/latest/python). For example, using the matrix `get_matrix_from_h5` from http://cf.10xgenomics.com/supp/cell-exp/megacell_tutorial-1.0.1.htm`: 
```
tmp = get_marix_from_h5(exprMat_path, 'dm6')
exprMat_full=pd.DataFrame(data=tmp.matrix.toarray().squeeze(),
                 index=tmp.gene_names.astype('str'),
                 columns=tmp.barcodes.astype('str')) 
del(tmp)
```

In [9]:
import pandas as pd
import numpy as np
import pyarrow.feather as feather

In [10]:
# From text file: 
exprMat = pd.read_csv(exprMat_path, sep='\t', index_col=0)

In [11]:
exprMat = exprMat.fillna(0)

In [12]:
exprMat.shape
exprMat.head()

(19972, 3005)

,1772071015_C02,1772071017_G12,1772071017_A05,1772071014_B06,1772067065_H06,1772071017_E02,1772067065_B07,1772067060_B09,1772071014_E04,1772071015_D04,...,1772066110_D12,1772071017_A07,1772063071_G10,1772058148_C03,1772063061_D09,1772067059_B04,1772066097_D04,1772063068_D01,1772066098_A12,1772058148_F03
cell_id,,,,,,,,,,,,,,,,,,,,,
Tspan12,0,0,0,3,0,0,3,0,0,0,...,0,0,0,0,0,0,0,0,0,1
Tshz1,3,1,0,2,2,2,2,1,0,2,...,0,0,0,0,0,0,0,0,0,1
Fnbp1l,3,1,6,4,1,2,1,0,5,2,...,0,0,0,0,0,0,0,0,0,0
Adamts15,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Cldn12,1,1,1,0,0,0,0,0,2,3,...,0,0,0,0,0,0,0,0,0,0


### Gene filter
Check whether these filters are appropriate for your dataset.
Adjust the minimum counts and number of cells if needed (e.g. `minCountsPerGene` and `minSamples`).

These are the same filters as the original [SCENIC implementation in R](https://github.com/aertslab/SCENIC) (expained in the tutorial ["Running SCENIC"](https://rawcdn.githack.com/aertslab/SCENIC/master/inst/doc/SCENIC_Running.html))

In [15]:
## General stats
nGenesPerCell = np.sum(exprMat>0, axis=0)
nCountsPerGene = np.sum(exprMat, axis=1)
nCellsPerGene = np.sum(exprMat>0, axis=1)
nCells=exprMat.shape[1]

print("General stats: ")
print("\t nGenes detected (counts>0):", np.sum(nCountsPerGene>0),)
print("\t nGenesPerCell: ", round(nGenesPerCell.min(),2), " - " , round(nGenesPerCell.max(),2))
print("\t Maximum value in matrix: ", np.nanmax(np.asmatrix(exprMat)))
print("\t Number of counts (in the dataset units) per gene:", nCountsPerGene.min(), " - " ,nCountsPerGene.max())
print("\t Number of cells in which each gene is detected:", nCellsPerGene.min(), " - " ,nCellsPerGene.max())

## Set thresholds
minCountsPerGene=3*.003*nCells # 3 counts in .3% of cells
minSamples=.003*nCells # .3% of cells

print("Used for filtering: ")
print("\t minCountsPerGene: ", round(minCountsPerGene,2))
print("\t minSamples: ", round(minSamples,2), " (nCells:",nCells,")")


print("Genes that pass the filter:")
# First filter
genesLeft_minReads=nCountsPerGene[nCountsPerGene>minCountsPerGene].index.tolist()
print("\t ", len(genesLeft_minReads), "\tgenes with counts per gene > ", round(minCountsPerGene,2))
  
# Second filter
nCellsPerGene2=nCellsPerGene[genesLeft_minReads]
genesLeft_minCells=nCellsPerGene2[nCellsPerGene2>minSamples].index.tolist()
print("\t ", len(genesLeft_minCells), "\tgenes detected in more than ", round(minSamples,2)," cells")

# In db
ctxDb_geneList=feather.read_feather(DATABASES_GLOB).columns[1:]
genesKept=list(set(genesLeft_minCells).intersection(ctxDb_geneList))
genesKept.sort()
print("\t ", len(genesKept), "\tgenes are in the motif database (the database contains info for",len(ctxDb_geneList),"genes)")

# Subst matrix
exprMat_filtered=exprMat.loc[genesKept]

General stats: 
	 nGenes detected (counts>0): 19972
	 nGenesPerCell:  770  -  8146
	 Maximum value in matrix:  10738
	 Number of counts (in the dataset units) per gene: 2  -  2200118
	 Number of cells in which each gene is detected: 1  -  3004
Used for filtering: 
	 minCountsPerGene:  27.05
	 minSamples:  9.02  (nCells: 3005 )
Genes that pass the filter:
	  15074 	genes with counts per gene >  27.05
	  15072 	genes detected in more than  9.02  cells
	  13800 	genes are in the motif database (the database contains info for  22058 genes)


In [23]:
exprMat.iloc[100:110,25:30]

,1772071017_F07,1772071017_E06,1772067066_C10,1772071017_B05,1772071014_E06
1700030J22Rik,0,1,0,0,0
1700030K09Rik,1,1,0,0,0
1700034H15Rik,0,0,0,0,0
1700037C18Rik,0,0,0,0,0
1700037H04Rik,2,4,3,4,5
1700047M11Rik,0,0,0,0,0
1700048O20Rik,0,0,0,0,3
1700049G17Rik,0,0,0,0,0
1700052K11Rik,3,0,0,0,0
1700052N19Rik,0,0,2,1,1


In [17]:
exprMat_filtered.shape

(13800, 3005)

### Save 
We suggest/recommend saving the filtered matrix as [.loom](https://linnarssonlab.org/loompy/) file:

In [18]:
import loompy
row_metadata={'Gene':list(exprMat_filtered.index)}
col_metadata={'CellID':list(exprMat_filtered.columns.values)}
loompy.create(exprMat_filtered_path, 
              exprMat_filtered.values,
              row_attrs=row_metadata, 
              col_attrs=col_metadata)

To reload/confirm that is saved propperly...

In [24]:
with loompy.connect(exprMat_filtered_path) as ds:
    # ds.ra.keys(); ds.ca.keys()
    exprMat=pd.DataFrame(ds[:,:], index=list(ds.ra['Gene']))
    exprMat.columns=list(ds.ca['CellID'])

exprMat.iloc[100:110,25:30]

,1772071017_F07,1772071017_E06,1772067066_C10,1772071017_B05,1772071014_E06
1700030J22Rik,0,1,0,0,0
1700030K09Rik,1,1,0,0,0
1700034H15Rik,0,0,0,0,0
1700037C18Rik,0,0,0,0,0
1700037H04Rik,2,4,3,4,5
1700047M11Rik,0,0,0,0,0
1700048O20Rik,0,0,0,0,3
1700049G17Rik,0,0,0,0,0
1700052K11Rik,3,0,0,0,0
1700052N19Rik,0,0,2,1,1


# Next:
Now you are ready to un SCENIC... 

Template notebook:`pySCENIC_template.ipynb`